In [20]:
import pandas as pd

data = pd.read_csv('used_cars_data.csv')
data = data.drop('S.No.', axis='columns')
data = data.drop('New_Price', axis='columns')

data.insert(3, 'Car_Age', 2024 - data['Year'])

mahindraData = data['Name'] == 'Mahindra E Verito D4'
data.loc[mahindraData, 'Mileage'] = '15.7 kmpl'
priusData = data['Name'] == 'Toyota Prius 2009-2016 Z4'
data.loc[priusData, 'Mileage'] = '18.62 kmpl'

data.to_csv('used_cars_data_modified.csv', encoding='utf-8', index=False)

1. To remove S.No. and New_Price columns, I first read the csv using pd.read_csv and set it to data. I then modified data using data.drop() and removed the 'S.No.' and 'New_Price' columns. 

To add the 'Car_Age' column, I used data.insert() and inserted this calculated data into column 3 so it would be next to the 'Year' column.

Next, I found the Electric vehicles by using Command F in the csv file and searching for 'electric'. The first row I found containing this was 'Mahindra E Verito D4,Chennai,2016,8,50000,Electric,Automatic,First,,72 CC,41 bhp,5.0,13.0'. I then googled the 2016 Mahindra E Verito D4 mileage. I found the value at https://www.carwale.com/mahindra-cars/verito/mileage/#:~:text=Mahindra%20Verito%20mileage%20starts%20at,Mahindra%20Verito%20is%2021%20kmpl to be 15.7 kmpl. This  site cites the Automotive Research Association of India as its source so I deemed it to be trustworthy.
The next Electric vehicle was 'Toyota Prius 2009-2016 Z4,Mumbai,2011,13,44000,Electric,Automatic,First,,1798 CC,73 bhp,5.0,12.75'. After Googling it, the average mpg of the car is 50. The source is https://www.kbb.com/toyota/prius/2011/ which I deemed to also be trustworthy as its the official Kelly Blue Book.
To try and be as accurate as possible, I decided to calculate how car mileage depreciates over time. I found from this source https://www.quora.com/How-much-does-gas-mileage-of-a-vehicle-degrade-over-time-If-a-new-vehicle-got-18-24-mpg-in-2013-how-much-is-it-likely-to-get-in-2018 that after 10 years, car mileage goes down approximately 10-15%. Taking the mean of this, we can say it loses 12.5% of its mileage which means the Mahindra E Verito D4 now gets 13.7375 kmpl or ~13.74 kmpl and the 2011 Prius now gets 43.8 mpg. Finally, we convert the Prius mileage to kmpl to match units with the other options. We get 18.62129kmpl or ~18.62 kmpl. This calculation was done using Google's Fuel Economy conversion tool.

Finally, to check that the output was correct, I wrote this modified data into another csv file called 'used_car_data_modified.csv' using data.to_csv() and checked it.

In [23]:
data = pd.read_csv('used_cars_data_modified.csv')
mileageList = data.get('Mileage').tolist()
mileageUnits = []
rmList = '1234567890. '
for x in mileageList:
  mileageUnits.append(''.join([i for i in x if i not in rmList]))
print(mileageUnits)
diffUnits = []
for x in mileageUnits:
  if x not in diffUnits:
    diffUnits.append(x)
print(diffUnits)

def convertMileage(mileage):
    value, unit = mileage.split(' ')
    value = float(value)
    
    if unit == 'km/kg':
        value = round(value / 0.775, 2)
    
    return value

data['Mileage'] = data['Mileage'].apply(convertMileage)
data.rename(columns={'Mileage': 'Mileage(kmpl)'}, inplace=True)
data.to_csv('used_cars_data_converted.csv', encoding='utf-8', index=False)

['km/kg', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'km/kg', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl', 'kmpl

2. First, I wrote code to check what units are within each list. In Mileage, I found both km/kg and kmpl were used. I decided to convert all values to kmpl which required multiplying all km/kg values by 1 kg / 0.775 liter to get all values in kmpl. (1 kg fuel = 0.775 liter value from https://ipshita.medium.com/run-through-of-eda-and-feature-engineering-8511d3a6f863#:~:text=Convert%20data%20to%20string,change%20it%20to%20km/l). 